In [ ]:
import pandas as pd
import plotly.express as px
from datetime import datetime

df = pd.read_csv("global_power_plant_database.csv")
usa = df[df["country"] == "USA"].copy()
usa["commissioning_year"] = pd.to_numeric(usa["commissioning_year"], errors="coerce")
usa = usa.dropna(subset=["latitude", "longitude", "capacity_mw"])
usa["age_years"] = datetime.now().year - usa["commissioning_year"]

# Build base figure with separate traces per fuel
fig = px.scatter_geo(
    usa,
    lat="latitude",
    lon="longitude",
    hover_name="name",
    hover_data={
        "capacity_mw": True,
        "age_years": True,
        "commissioning_year": True,
    },
    size="capacity_mw",
    color="primary_fuel",
    size_max=20,
    opacity=0.75,
    projection="albers usa",
)

# All unique fuels
fuels = usa["primary_fuel"].unique().tolist()

# Create dropdown buttons
buttons = [
    dict(
        label="All Fuels",
        method="update",
        args=[{"visible": [True] * len(fig.data)}],
    )
]

# One button per fuel type
for i, fuel in enumerate(fuels):
    visibility = [False] * len(fig.data)
    visibility[i] = True       # show only this fuel’s trace
    buttons.append(
        dict(
            label=fuel,
            method="update",
            args=[{"visible": visibility}],
        )
    )

fig.update_layout(
    width=2000,
    height=1200,
    title="US Power Plants – Filterable by Fuel Type",
    geo=dict(scope="usa"),
    updatemenus=[
        dict(
            type="dropdown",
            x=1.15,
            y=1,
            showactive=True,
            buttons=buttons,
        )
    ],
    legend=dict(
        title="Fuel Type",
        bgcolor="rgba(255,255,255,0.5)",
        bordercolor="black",
        borderwidth=1
    )
)

fig.show()


In [ ]:
import pandas as pd
import plotly.graph_objects as go
from datetime import datetime
import numpy as np

# --------------------------------------------------
# Load & clean data
# --------------------------------------------------
df = pd.read_csv("global_power_plant_database.csv")

# Filter to USA
usa = df[df["country"] == "USA"].copy()

# Ensure numeric year and drop rows without essentials
usa["commissioning_year"] = pd.to_numeric(usa["commissioning_year"], errors="coerce")
usa = usa.dropna(subset=["latitude", "longitude", "capacity_mw", "commissioning_year"])

# Age for hover (optional)
current_year = datetime.now().year
usa["age_years"] = current_year - usa["commissioning_year"]

# --------------------------------------------------
# Top 20 largest plants (labels)
# --------------------------------------------------
top20_idx = usa.sort_values("capacity_mw", ascending=False).head(20).index
usa["top_label"] = ""
usa.loc[top20_idx, "top_label"] = usa.loc[top20_idx, "name"]

# --------------------------------------------------
# Year range & slider steps (every 5 years)
# --------------------------------------------------
min_year = int(usa["commissioning_year"].min())
max_year = int(usa["commissioning_year"].max())

# 5-year steps; ensure we include max_year
years = list(range(min_year, max_year + 1, 5))
if years[-1] != max_year:
    years.append(max_year)

# Precompute max capacity for marker scaling
max_cap = usa["capacity_mw"].max()

def make_frame_subset(year):
    """Return subset of plants commissioned up to and including 'year'."""
    sub = usa[usa["commissioning_year"] <= year].copy()
    if sub.empty:
        return sub, []
    # Size scaling: 5–40 using sqrt scaling
    sizes = 5 + 35 * np.sqrt(sub["capacity_mw"] / max_cap)
    return sub, sizes

# --------------------------------------------------
# Initial frame (first year in slider)
# --------------------------------------------------
initial_year = years[0]
sub0, sizes0 = make_frame_subset(initial_year)

fig = go.Figure()

fig.add_trace(
    go.Scattergeo(
        lon=sub0["longitude"],
        lat=sub0["latitude"],
        text=sub0["top_label"],  # labels for top 20 (blank for others)
        hovertext=(
            "Name: " + sub0["name"]
            + "<br>Fuel: " + sub0["primary_fuel"].astype(str)
            + "<br>Capacity (MW): " + sub0["capacity_mw"].round(1).astype(str)
            + "<br>Year: " + sub0["commissioning_year"].astype(int).astype(str)
            + "<br>Age (yrs): " + sub0["age_years"].astype(int).astype(str)
        ),
        mode="markers+text",
        marker=dict(
            size=sizes0,
            opacity=0.75,
            line=dict(width=0.2, color="black"),
            # Color each plant by its primary_fuel
            # Use 'marker.color' as array + 'marker.colorscale' if you want numeric mapping,
            # but for categorical fuel we use 'marker.color=None' here and use 'showlegend=False'
            # and rely on hover text. If you want a full categorical legend by fuel, we can
            # switch to one-trace-per-fuel; for now this keeps things simple.
            color="blue",  # single color; see note below to add per-fuel colors
        ),
        textposition="top center",
        name=str(initial_year),
        showlegend=False,
    )
)

# NOTE:
# If you still want different colors per primary_fuel *and* a legend,
# we can build multiple traces per fuel. For cumulative + slider it's
# a bit more code, so I'm keeping it simple here. Happy to extend.

# --------------------------------------------------
# Build animation frames (cumulative up to each year)
# --------------------------------------------------
frames = []
for year in years:
    sub, sizes = make_frame_subset(year)
    frames.append(
        go.Frame(
            data=[
                go.Scattergeo(
                    lon=sub["longitude"],
                    lat=sub["latitude"],
                    text=sub["top_label"],
                    hovertext=(
                        "Name: " + sub["name"]
                        + "<br>Fuel: " + sub["primary_fuel"].astype(str)
                        + "<br>Capacity (MW): " + sub["capacity_mw"].round(1).astype(str)
                        + "<br>Year: " + sub["commissioning_year"].astype(int).astype(str)
                        + "<br>Age (yrs): " + sub["age_years"].astype(int).astype(str)
                    ),
                    mode="markers+text",
                    marker=dict(
                        size=sizes if len(sizes) > 0 else [],
                        opacity=0.75,
                        line=dict(width=0.2, color="black"),
                        color="blue",  # same note as above
                    ),
                    textposition="top center",
                    name=str(year),
                    showlegend=False,
                )
            ],
            name=str(year),
        )
    )

fig.frames = frames

# --------------------------------------------------
# Slider configuration: integer labels, 5-year steps
# --------------------------------------------------
slider_steps = []
for year in years:
    slider_steps.append(
        dict(
            method="animate",
            args=[
                [str(year)],
                dict(
                    mode="immediate",
                    frame=dict(duration=0, redraw=True),
                    transition=dict(duration=0),
                ),
            ],
            label=str(year),  # integer label
        )
    )

sliders = [
    dict(
        active=0,
        currentvalue={"prefix": "Commissioning year ≤ ", "font": {"size": 16}},
        pad={"t": 50},
        steps=slider_steps,
    )
]

# --------------------------------------------------
# Layout
# --------------------------------------------------
fig.update_layout(
    width=1200,
    height=720,
    title="US Power Plants – Cumulative Commissioning by Year (All Plants up to Selected Year)",
    geo=dict(
        scope="usa",
        projection_type="albers usa",
        landcolor="rgb(240,240,240)",
        lakecolor="rgb(220,220,220)",
        bgcolor="rgba(0,0,0,0)",
    ),
    sliders=sliders,
    updatemenus=[
        dict(
            type="buttons",
            showactive=False,
            x=0.1,
            y=0,
            xanchor="right",
            yanchor="top",
            pad={"t": 0, "r": 10},
            buttons=[
                dict(
                    label="Play",
                    method="animate",
                    args=[
                        None,
                        dict(
                            frame=dict(duration=500, redraw=True),
                            transition=dict(duration=0),
                            fromcurrent=True,
                            mode="immediate",
                        ),
                    ],
                ),
                dict(
                    label="Pause",
                    method="animate",
                    args=[
                        [None],
                        dict(
                            frame=dict(duration=0, redraw=False),
                            transition=dict(duration=0),
                            mode="immediate",
                        ),
                    ],
                ),
            ],
        )
    ],
)

fig.show()

# --------------------------------------------------
# Optional: export first frame as PNG/PDF (requires kaleido)
# --------------------------------------------------
# fig.write_image("us_power_plants_cumulative.png", scale=2)
# fig.write_image("us_power_plants_cumulative.pdf")


In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import plotly.graph_objects as go
import plotly.express as px

# --------------------------------------------------
# Load & clean data
# --------------------------------------------------
df = pd.read_csv("global_power_plant_database.csv")

# Filter to USA
usa = df[df["country"] == "USA"].copy()

# Ensure numeric year and drop rows without essentials
usa["commissioning_year"] = pd.to_numeric(usa["commissioning_year"], errors="coerce")
usa = usa.dropna(subset=["latitude", "longitude", "capacity_mw", "commissioning_year"])

# Age for hover (optional)
current_year = datetime.now().year
usa["age_years"] = current_year - usa["commissioning_year"]

# --------------------------------------------------
# Top 20 largest plants (labels)
# --------------------------------------------------
top20_idx = usa.sort_values("capacity_mw", ascending=False).head(20).index
usa["top_label"] = ""
usa.loc[top20_idx, "top_label"] = usa.loc[top20_idx, "name"]

# --------------------------------------------------
# Primary fuel list & color map (consistent across frames)
# --------------------------------------------------
fuels = sorted(usa["primary_fuel"].dropna().unique())

# Qualitative color palette (loop if needed)
palette = (
    px.colors.qualitative.Set2
    + px.colors.qualitative.Set1
    + px.colors.qualitative.Dark24
)
color_map = {fuel: palette[i % len(palette)] for i, fuel in enumerate(fuels)}

# --------------------------------------------------
# Year range & slider steps (every 5 years, integer labels)
# --------------------------------------------------
min_year = int(usa["commissioning_year"].min())
max_year = int(usa["commissioning_year"].max())

years = list(range(min_year, max_year + 1, 5))
if years[-1] != max_year:
    years.append(max_year)

# Capacity scaling (max bubble size ≈ 20)
max_cap = usa["capacity_mw"].max()

def subset_by_year_and_fuel(year, fuel):
    """Return plants of a given fuel commissioned <= year."""
    sub = usa[
        (usa["commissioning_year"] <= year)
        & (usa["primary_fuel"] == fuel)
    ].copy()
    if sub.empty:
        return sub, []
    # Size scaling: 5–20 using sqrt scaling
    sizes = 5 + 15 * np.sqrt(sub["capacity_mw"] / max_cap)
    return sub, sizes

# --------------------------------------------------
# Initial traces (for first slider year)
# --------------------------------------------------
initial_year = years[0]
fig = go.Figure()

for fuel in fuels:
    sub, sizes = subset_by_year_and_fuel(initial_year, fuel)
    fig.add_trace(
        go.Scattergeo(
            lon=sub["longitude"],
            lat=sub["latitude"],
            text=sub["top_label"],  # labels for top 20 (others blank)
            hovertext=(
                "Name: " + sub["name"]
                + "<br>Fuel: " + sub["primary_fuel"].astype(str)
                + "<br>Capacity (MW): " + sub["capacity_mw"].round(1).astype(str)
                + "<br>Year: " + sub["commissioning_year"].astype(int).astype(str)
                + "<br>Age (yrs): " + sub["age_years"].astype(int).astype(str)
            ) if not sub.empty else None,
            mode="markers+text",
            marker=dict(
                size=sizes,
                opacity=0.75,
                line=dict(width=0.2, color="black"),
                color=color_map[fuel],
            ),
            textposition="top center",
            name=fuel,      # legend entry
            showlegend=True,
        )
    )

# --------------------------------------------------
# Animation frames (cumulative by year)
# --------------------------------------------------
frames = []
for year in years:
    frame_traces = []
    for fuel in fuels:
        sub, sizes = subset_by_year_and_fuel(year, fuel)
        frame_traces.append(
            go.Scattergeo(
                lon=sub["longitude"],
                lat=sub["latitude"],
                text=sub["top_label"],
                hovertext=(
                    "Name: " + sub["name"]
                    + "<br>Fuel: " + sub["primary_fuel"].astype(str)
                    + "<br>Capacity (MW): " + sub["capacity_mw"].round(1).astype(str)
                    + "<br>Year: " + sub["commissioning_year"].astype(int).astype(str)
                    + "<br>Age (yrs): " + sub["age_years"].astype(int).astype(str)
                ) if not sub.empty else None,
                mode="markers+text",
                marker=dict(
                    size=sizes,
                    opacity=0.75,   # base opacity; will be overridden by opacity controls
                    line=dict(width=0.2, color="black"),
                    color=color_map[fuel],
                ),
                textposition="top center",
                name=fuel,
                showlegend=False,  # legend comes from initial traces
            )
        )
    frames.append(go.Frame(data=frame_traces, name=str(year)))

fig.frames = frames

# --------------------------------------------------
# Slider (integer labels, 5-year steps)
# --------------------------------------------------
slider_steps = []
for year in years:
    slider_steps.append(
        dict(
            method="animate",
            args=[
                [str(year)],
                dict(
                    mode="immediate",
                    frame=dict(duration=0, redraw=True),
                    transition=dict(duration=0),
                ),
            ],
            label=str(year),
        )
    )

sliders = [
    dict(
        active=0,
        currentvalue={"prefix": "Commissioning year ≤ ", "font": {"size": 16}},
        pad={"t": 50},
        steps=slider_steps,
    )
]

# --------------------------------------------------
# Visibility controls (checkbox-style via buttons + legend)
#   - Show All Fuels
#   - Hide All Fuels
#   (You can still click legend entries to toggle each fuel on/off)
# --------------------------------------------------
visibility_buttons = [
    dict(
        label="Show All Fuels",
        method="update",
        args=[{"visible": [True] * len(fuels)}],
    ),
    dict(
        label="Hide All Fuels",
        method="update",
        args=[{"visible": [False] * len(fuels)}],
    ),
]

# --------------------------------------------------
# Per-fuel opacity controls (highlight one fuel)
#   - Reset opacity
#   - Highlight <fuel> (sets that fuel to 1.0, all others to 0.2)
# --------------------------------------------------
opacity_buttons = []

# Reset opacity button
opacity_buttons.append(
    dict(
        label="Reset Opacity",
        method="update",
        args=[{
            "marker": [dict(opacity=0.75) for _ in range(len(fuels))]
        }],
    )
)

# One button per fuel to highlight it
for i, fuel in enumerate(fuels):
    opacities = [0.2] * len(fuels)
    opacities[i] = 1.0
    opacity_buttons.append(
        dict(
            label=f"Highlight {fuel}",
            method="update",
            args=[{
                "marker": [dict(opacity=o) for o in opacities]
            }],
        )
    )

# --------------------------------------------------
# Animation speed controls (single menu, multiple play speeds)
# --------------------------------------------------
speed_buttons = [
    dict(
        label="Play (Slow)",
        method="animate",
        args=[
            None,
            dict(
                frame=dict(duration=1000, redraw=True),
                transition=dict(duration=0),
                fromcurrent=True,
                mode="immediate",
            ),
        ],
    ),
    dict(
        label="Play (Normal)",
        method="animate",
        args=[
            None,
            dict(
                frame=dict(duration=500, redraw=True),
                transition=dict(duration=0),
                fromcurrent=True,
                mode="immediate",
            ),
        ],
    ),
    dict(
        label="Play (Fast)",
        method="animate",
        args=[
            None,
            dict(
                frame=dict(duration=200, redraw=True),
                transition=dict(duration=0),
                fromcurrent=True,
                mode="immediate",
            ),
        ],
    ),
    dict(
        label="Pause",
        method="animate",
        args=[
            [None],
            dict(
                frame=dict(duration=0, redraw=False),
                transition=dict(duration=0),
                mode="immediate",
            ),
        ],
    ),
]

# --------------------------------------------------
# Updatemenus (3 menus: visibility, opacity, animation speed)
# --------------------------------------------------
updatemenus = [
    # Visibility (checkbox-style via Show All / Hide All + legend toggles)
    dict(
        type="buttons",
        direction="down",
        x=1.18,
        y=1.0,
        showactive=True,
        buttons=visibility_buttons,
        xanchor="left",
        yanchor="top",
    ),
    # Per-fuel opacity controls
    dict(
        type="buttons",
        direction="down",
        x=1.18,
        y=0.75,
        showactive=True,
        buttons=opacity_buttons,
        xanchor="left",
        yanchor="top",
    ),
    # Animation speed controls
    dict(
        type="buttons",
        direction="right",
        x=0.0,
        y=0.0,
        xanchor="left",
        yanchor="top",
        showactive=False,
        buttons=speed_buttons,
    ),
]

# --------------------------------------------------
# Layout (1200 × 720)
# --------------------------------------------------
fig.update_layout(
    width=1200,
    height=720,
    title="US Power Plants – Cumulative Commissioning by Year (Per-Fuel Colors)",
    geo=dict(
        scope="usa",
        projection_type="albers usa",
        landcolor="rgb(240,240,240)",
        lakecolor="rgb(220,220,220)",
        bgcolor="rgba(0,0,0,0)",
    ),
    sliders=sliders,
    updatemenus=updatemenus,
    legend=dict(
        title="Primary Fuel",
        bgcolor="rgba(255,255,255,0.7)",
        bordercolor="black",
        borderwidth=1,
    ),
)

fig.show()

# --------------------------------------------------
# Optional: export static image (first frame) to PNG/PDF (requires kaleido)
# --------------------------------------------------
fig.write_image("us_power_plants_cumulative_fuel.png", scale=2)
fig.write_image("us_power_plants_cumulative_fuel.pdf")


In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import plotly.graph_objects as go
import plotly.express as px

# --------------------------------------------------
# Load & clean data
# --------------------------------------------------
df = pd.read_csv("global_power_plant_database.csv")

# Filter to USA
usa = df[df["country"] == "USA"].copy()

# Ensure numeric year and drop rows without essentials
usa["commissioning_year"] = pd.to_numeric(usa["commissioning_year"], errors="coerce")
usa = usa.dropna(subset=["latitude", "longitude", "capacity_mw", "commissioning_year"])

# Age for hover (optional)
current_year = datetime.now().year
usa["age_years"] = current_year - usa["commissioning_year"]

# --------------------------------------------------
# Top 20 largest plants (labels)
# --------------------------------------------------
top20_idx = usa.sort_values("capacity_mw", ascending=False).head(20).index
usa["top_label"] = ""
usa.loc[top20_idx, "top_label"] = usa.loc[top20_idx, "name"]

# --------------------------------------------------
# Primary fuel list & color map (consistent across frames)
# --------------------------------------------------
fuels = sorted(usa["primary_fuel"].dropna().unique())

# Qualitative color palette (loop if needed)
palette = (
    px.colors.qualitative.Set2
    + px.colors.qualitative.Set1
    + px.colors.qualitative.Dark24
)
color_map = {fuel: palette[i % len(palette)] for i, fuel in enumerate(fuels)}

# --------------------------------------------------
# Year range & slider steps (every 5 years, integer labels)
# --------------------------------------------------
min_year = int(usa["commissioning_year"].min())
max_year = int(usa["commissioning_year"].max())

years = list(range(min_year, max_year + 1, 5))
if years[-1] != max_year:
    years.append(max_year)

# Capacity scaling (max bubble size ≈ 20)
max_cap = usa["capacity_mw"].max()

def subset_by_year_and_fuel(year, fuel):
    """Return plants of a given fuel commissioned <= year."""
    sub = usa[
        (usa["commissioning_year"] <= year)
        & (usa["primary_fuel"] == fuel)
    ].copy()
    if sub.empty:
        return sub, []
    # Size scaling: 5–20 using sqrt scaling
    sizes = 5 + 15 * np.sqrt(sub["capacity_mw"] / max_cap)
    return sub, sizes

# --------------------------------------------------
# Initial traces (for first slider year)
# --------------------------------------------------
initial_year = years[0]
fig = go.Figure()

for fuel in fuels:
    sub, sizes = subset_by_year_and_fuel(initial_year, fuel)
    fig.add_trace(
        go.Scattergeo(
            lon=sub["longitude"],
            lat=sub["latitude"],
            text=sub["top_label"],  # labels for top 20 (others blank)
            hovertext=(
                "Name: " + sub["name"]
                + "<br>Fuel: " + sub["primary_fuel"].astype(str)
                + "<br>Capacity (MW): " + sub["capacity_mw"].round(1).astype(str)
                + "<br>Year: " + sub["commissioning_year"].astype(int).astype(str)
                + "<br>Age (yrs): " + sub["age_years"].astype(int).astype(str)
            ) if not sub.empty else None,
            mode="markers+text",
            marker=dict(
                size=sizes,
                opacity=0.75,
                line=dict(width=0.2, color="black"),
                color=color_map[fuel],
            ),
            textposition="top center",
            name=fuel,      # legend entry (this is what you click to show/hide)
            showlegend=True,
        )
    )

# --------------------------------------------------
# Animation frames (cumulative by year)
# --------------------------------------------------
frames = []
for year in years:
    frame_traces = []
    for fuel in fuels:
        sub, sizes = subset_by_year_and_fuel(year, fuel)
        frame_traces.append(
            go.Scattergeo(
                lon=sub["longitude"],
                lat=sub["latitude"],
                text=sub["top_label"],
                hovertext=(
                    "Name: " + sub["name"]
                    + "<br>Fuel: " + sub["primary_fuel"].astype(str)
                    + "<br>Capacity (MW): " + sub["capacity_mw"].round(1).astype(str)
                    + "<br>Year: " + sub["commissioning_year"].astype(int).astype(str)
                    + "<br>Age (yrs): " + sub["age_years"].astype(int).astype(str)
                ) if not sub.empty else None,
                mode="markers+text",
                marker=dict(
                    size=sizes,
                    opacity=0.75,
                    line=dict(width=0.2, color="black"),
                    color=color_map[fuel],
                ),
                textposition="top center",
                name=fuel,
                showlegend=False,  # legend comes from initial traces
            )
        )
    frames.append(go.Frame(data=frame_traces, name=str(year)))

fig.frames = frames

# --------------------------------------------------
# Slider (integer labels, 5-year steps, cumulative)
# --------------------------------------------------
slider_steps = []
for year in years:
    slider_steps.append(
        dict(
            method="animate",
            args=[
                [str(year)],
                dict(
                    mode="immediate",
                    frame=dict(duration=0, redraw=True),
                    transition=dict(duration=0),
                ),
            ],
            label=str(year),
        )
    )

sliders = [
    dict(
        active=0,
        currentvalue={"prefix": "Commissioning year ≤ ", "font": {"size": 16}},
        pad={"t": 50},
        steps=slider_steps,
    )
]

# --------------------------------------------------
# Animation speed controls
# --------------------------------------------------
speed_buttons = [
    dict(
        label="Play (Slow)",
        method="animate",
        args=[
            None,
            dict(
                frame=dict(duration=1000, redraw=True),
                transition=dict(duration=0),
                fromcurrent=True,
                mode="immediate",
            ),
        ],
    ),
    dict(
        label="Play (Normal)",
        method="animate",
        args=[
            None,
            dict(
                frame=dict(duration=500, redraw=True),
                transition=dict(duration=0),
                fromcurrent=True,
                mode="immediate",
            ),
        ],
    ),
    dict(
        label="Play (Fast)",
        method="animate",
        args=[
            None,
            dict(
                frame=dict(duration=200, redraw=True),
                transition=dict(duration=0),
                fromcurrent=True,
                mode="immediate",
            ),
        ],
    ),
    dict(
        label="Pause",
        method="animate",
        args=[
            [None],
            dict(
                frame=dict(duration=0, redraw=False),
                transition=dict(duration=0),
                mode="immediate",
            ),
        ],
    ),
]

updatemenus = [
    dict(
        type="buttons",
        direction="right",
        x=0.0,
        y=0.0,
        xanchor="left",
        yanchor="top",
        showactive=False,
        buttons=speed_buttons,
    ),
]

# --------------------------------------------------
# Layout (1200 × 720, legend inside the figure area)
# --------------------------------------------------
fig.update_layout(
    width=1200,
    height=720,
    title="US Power Plants – Cumulative Commissioning by Year (Per-Fuel Colors)",
    geo=dict(
        scope="usa",
        projection_type="albers usa",
        landcolor="rgb(240,240,240)",
        lakecolor="rgb(220,220,220)",
        bgcolor="rgba(0,0,0,0)",
    ),
    sliders=sliders,
    updatemenus=updatemenus,
    legend=dict(
        title="Primary Fuel",
        orientation="v",
        x=0.99,
        xanchor="right",
        y=0.99,
        yanchor="top",
        bgcolor="rgba(255,255,255,0.8)",
        bordercolor="black",
        borderwidth=1,
    ),
    margin=dict(l=50, r=200, t=80, b=60),  # extra right margin so legend is visible
)

fig.show()

# --------------------------------------------------
# Optional: export static image (first frame) to PNG/PDF (requires kaleido)
# --------------------------------------------------
# fig.write_image("us_power_plants_cumulative_fuel.png", scale=2)
# fig.write_image("us_power_plants_cumulative_fuel.pdf")


In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import plotly.graph_objects as go
import plotly.express as px

# --------------------------------------------------
# Load & clean data
# --------------------------------------------------
df = pd.read_csv("global_power_plant_database.csv")

# Filter to USA
usa = df[df["country"] == "USA"].copy()

# Ensure numeric year and drop rows without essentials
usa["commissioning_year"] = pd.to_numeric(usa["commissioning_year"], errors="coerce")
usa = usa.dropna(subset=["latitude", "longitude", "capacity_mw", "commissioning_year"])

# Age for hover (optional)
current_year = datetime.now().year
usa["age_years"] = current_year - usa["commissioning_year"]

# --------------------------------------------------
# Top 20 largest plants (labels)
# --------------------------------------------------
top20_idx = usa.sort_values("capacity_mw", ascending=False).head(20).index
usa["top_label"] = ""
usa.loc[top20_idx, "top_label"] = usa.loc[top20_idx, "name"]

# --------------------------------------------------
# Primary fuel list & color map
# --------------------------------------------------
fuels = sorted(usa["primary_fuel"].dropna().unique())

palette = (
    px.colors.qualitative.Set2
    + px.colors.qualitative.Set1
    + px.colors.qualitative.Dark24
)
color_map = {fuel: palette[i % len(palette)] for i, fuel in enumerate(fuels)}

# --------------------------------------------------
# Size scaling (max ~20)
# --------------------------------------------------
max_cap = usa["capacity_mw"].max()
bubble_sizes = 5 + 15 * np.sqrt(usa["capacity_mw"] / max_cap)

# --------------------------------------------------
# Build static figure (no slider, no animation)
# --------------------------------------------------
fig = go.Figure()

for fuel in fuels:
    sub = usa[usa["primary_fuel"] == fuel].copy()
    sizes = 5 + 15 * np.sqrt(sub["capacity_mw"] / max_cap)

    fig.add_trace(
        go.Scattergeo(
            lon=sub["longitude"],
            lat=sub["latitude"],
            text=sub["top_label"],
            hovertext=(
                "Name: " + sub["name"]
                + "<br>Fuel: " + sub["primary_fuel"].astype(str)
                + "<br>Capacity (MW): " + sub["capacity_mw"].round(1).astype(str)
                + "<br>Year: " + sub["commissioning_year"].astype(int).astype(str)
                + "<br>Age (yrs): " + sub["age_years"].astype(int).astype(str)
            ),
            mode="markers+text",
            marker=dict(
                size=sizes,
                color=color_map[fuel],
                opacity=0.75,
                line=dict(width=0.2, color="black")
            ),
            textposition="top center",
            name=fuel,
            showlegend=True,
        )
    )

# --------------------------------------------------
# Animation speed controls removed (no animation)
# --------------------------------------------------
updatemenus = []  # nothing here now

# --------------------------------------------------
# Layout
# --------------------------------------------------
fig.update_layout(
    width=1200,
    height=720,
    title="US Power Plants – All Plants (Per-Fuel Colors)",
    geo=dict(
        scope="usa",
        projection_type="albers usa",
        landcolor="rgb(240,240,240)",
        lakecolor="rgb(220,220,220)",
        bgcolor="rgba(0,0,0,0)",
    ),
    updatemenus=updatemenus,
    legend=dict(
        title="Primary Fuel",
        orientation="v",
        x=0.99,
        xanchor="right",
        y=0.99,
        yanchor="top",
        bgcolor="rgba(255,255,255,0.8)",
        bordercolor="black",
        borderwidth=1,
    ),
    margin=dict(l=50, r=200, t=80, b=60),
)

fig.show()

# --------------------------------------------------
# Optional export
# --------------------------------------------------
# fig.write_image("us_power_plants_static.png", scale=2)
# fig.write_image("us_power_plants_static.pdf")


In [ ]:
pip install dash

In [6]:
import pandas as pd
import numpy as np
from datetime import datetime

import plotly.graph_objects as go
import plotly.express as px

from dash import Dash, dcc, html
from dash.dependencies import Input, Output

# --------------------------------------------------
# Load & preprocess data
# --------------------------------------------------
df = pd.read_csv("global_power_plant_database.csv")

# Filter to USA
usa = df[df["country"] == "USA"].copy()

# Ensure numeric year and drop rows without essentials
usa["commissioning_year"] = pd.to_numeric(usa["commissioning_year"], errors="coerce")
usa = usa.dropna(subset=["latitude", "longitude", "capacity_mw", "commissioning_year"])

# Age for hover (optional)
current_year = datetime.now().year
usa["age_years"] = current_year - usa["commissioning_year"]

# Fuel list
fuels = sorted(usa["primary_fuel"].dropna().unique())

# Color map per fuel (consistent)
palette = (
    px.colors.qualitative.Set2
    + px.colors.qualitative.Set1
    + px.colors.qualitative.Dark24
)
color_map = {fuel: palette[i % len(palette)] for i, fuel in enumerate(fuels)}

# Year range for slider
min_year = int(usa["commissioning_year"].min())
max_year = int(usa["commissioning_year"].max())

# Capacity range for MW filter
min_cap = float(usa["capacity_mw"].min())
max_cap = float(usa["capacity_mw"].max())

# --------------------------------------------------
# Dash app
# --------------------------------------------------
app = Dash(__name__)

app.layout = html.Div(
    style={"font-family": "Arial, sans-serif", "padding": "10px"},
    children=[
        html.H2("US Power Plants – Interactive Map"),

        html.Div(
            style={"display": "flex", "gap": "30px", "flex-wrap": "wrap"},
            children=[
                # Fuel multi-select dropdown
                html.Div(
                    style={"minWidth": "250px"},
                    children=[
                        html.Label("Fuel types to display:"),
                        dcc.Dropdown(
                            id="fuel-dropdown",
                            options=[{"label": f, "value": f} for f in fuels],
                            value=fuels,         # default: all fuels selected
                            multi=True,
                            placeholder="Select one or more fuels",
                        ),
                    ],
                ),

                # Year range slider (twice as long, sparser marks)
html.Div(
    style={"minWidth": "350px", "width": "700px"},  # make the container wide
    children=[
        html.Label("Commissioning year range:"),
        dcc.RangeSlider(
            id="year-range",
            min=min_year,
            max=max_year,
            step=1,
            value=[min_year, max_year],
            marks={
                y: str(y)
                for y in range(
                    (min_year // 10) * 10,
                    max_year + 1,
                    10      # marks every 10 years -> fewer labels
                )
            },
            allowCross=False,
            tooltip={"placement": "bottom", "always_visible": False},
            # NOTE: no style= here, style lives on the wrapping Div
        ),
    ],
),

                # Minimum MW filter
                html.Div(
                    style={"minWidth": "250px"},
                    children=[
                        html.Label("Minimum capacity (MW):"),
                        dcc.Slider(
                            id="min-mw",
                            min=0,
                            max=max_cap,
                            step=50,
                            value=0,
                            tooltip={"placement": "bottom", "always_visible": False},
                        ),
                        html.Div(
                            id="min-mw-label",
                            style={"marginTop": "5px", "fontSize": "0.9em"},
                        ),
                    ],
                ),
            ],
        ),

        html.Hr(),

        dcc.Graph(
            id="plant-map",
            style={"width": "1200px", "height": "720px"},
        ),
    ],
)

# --------------------------------------------------
# Callback to update MW label
# --------------------------------------------------
@app.callback(
    Output("min-mw-label", "children"),
    Input("min-mw", "value"),
)
def update_min_mw_label(min_mw):
    return f"Showing plants with capacity ≥ {min_mw:.0f} MW"


# --------------------------------------------------
# Callback to update the map
# --------------------------------------------------
@app.callback(
    Output("plant-map", "figure"),
    [
        Input("fuel-dropdown", "value"),
        Input("year-range", "value"),
        Input("min-mw", "value"),
    ],
)
def update_map(selected_fuels, year_range, min_mw):
    # Handle empty selection (show nothing)
    if not selected_fuels:
        filtered = usa.iloc[0:0].copy()
    else:
        start_year, end_year = year_range
        filtered = usa[
            (usa["primary_fuel"].isin(selected_fuels))
            & (usa["commissioning_year"] >= start_year)
            & (usa["commissioning_year"] <= end_year)
            & (usa["capacity_mw"] >= min_mw)
        ].copy()

    # If no plants after filtering, create an empty figure with same layout
    if filtered.empty:
        fig = go.Figure()
        fig.update_layout(
            width=1200,
            height=720,
            title="No plants match the current filters",
            geo=dict(
                scope="usa",
                projection_type="albers usa",
                landcolor="rgb(240,240,240)",
                lakecolor="rgb(220,220,220)",
                bgcolor="rgba(0,0,0,0)",
            ),
        )
        return fig

    # --- NO top-20 labels anymore ---
    # We just plot markers; no text labels.
    # Bubble size scaling (max ~20) within filtered subset
    max_cap_filtered = float(filtered["capacity_mw"].max())

    def size_scale(cap_series):
        return 5 + 15 * np.sqrt(cap_series / max_cap_filtered)

    fig = go.Figure()

    for fuel in sorted(filtered["primary_fuel"].dropna().unique()):
        sub = filtered[filtered["primary_fuel"] == fuel].copy()
        sizes = size_scale(sub["capacity_mw"])

        fig.add_trace(
            go.Scattergeo(
                lon=sub["longitude"],
                lat=sub["latitude"],
                hovertext=(
                    "Name: " + sub["name"]
                    + "<br>Fuel: " + sub["primary_fuel"].astype(str)
                    + "<br>Capacity (MW): " + sub["capacity_mw"].round(1).astype(str)
                    + "<br>Year: " + sub["commissioning_year"].astype(int).astype(str)
                    + "<br>Age (yrs): " + sub["age_years"].astype(int).astype(str)
                ),
                mode="markers",     # <-- markers only (no text labels)
                marker=dict(
                    size=sizes,
                    color=color_map.get(fuel, "gray"),
                    opacity=0.75,
                    line=dict(width=0.2, color="black"),
                ),
                name=fuel,
                showlegend=True,
            )
        )

    fig.update_layout(
        width=1200,
        height=720,
        title="US Power Plants – Filtered by Fuel, Year, and Capacity",
        geo=dict(
            scope="usa",
            projection_type="albers usa",
            landcolor="rgb(240,240,240)",
            lakecolor="rgb(220,220,220)",
            bgcolor="rgba(0,0,0,0)",
        ),
        legend=dict(
            title="Primary Fuel",
            orientation="v",
            x=0.99,
            xanchor="right",
            y=0.99,
            yanchor="top",
            bgcolor="rgba(255,255,255,0.8)",
            bordercolor="black",
            borderwidth=1,
        ),
        margin=dict(l=50, r=200, t=80, b=60),
    )

    return fig


if __name__ == "__main__":
    app.run(debug=True)
